# Image Segmentation Predictor

For several testareas, with georeferencing

# Libraries

In [1]:
from fastai.vision.all import *
import pandas as pd
import gc # garbage collector
import os
from tqdm import tqdm
import rasterio as rio

In [2]:
# for docker compatibility in Ubuntu (pytorch bug)
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

In [3]:
# everything happens in 'path' (except the model file)
path = Path("../Data/")

#### Choose class and other parameters:

In [4]:
#VERSION = "C46" # BASELINE
#VERSION = "C83" # new BASELINE
VERSION = "C99b" # new BASELINE

VERSION = "D127b"
VERSION = "C105c"
VERSION = "C108"

#MERGE_AREAS = False
MERGE_AREAS = True

CONVERT_MASK = False
#CONVERT_MASK = True # NO LONGER REQUIRED!?!

In [5]:
# should we use Nvidia CUDA?
#use_CUDA = False
use_CUDA = True

# what class(es) are we predicting?
myclass = "reservoir"

# use TTA (test time augmentation)? Does not work with my Deeplab implementation though.
use_TTA = False
#use_TTA = True

############## TODO ### 
# where is my pre-trained model?
learner_path = "./models/"

# nnet model file name
nnet = "m_*" + VERSION + "_"
#nnet = "m_d*" + VERSION + "_"
#nnet = "m_h*" + VERSION + "_"
#nnet = 'stage-1_*'+ VERSION + "_"

myclass, learner_path, nnet

('reservoir', './models/', 'm_*C108_')

In [6]:
# path to images, i.e. which ones should be predicted?
image_path_list = [  
    #[path/'Deeplearningtrialarea2/BingImages_zoom_19',      path/'Deeplearningtrialarea2/PredictedMasks'],
    ####[path/'Deeplearningtrialarea2_overlap/BingImages',      path/'Deeplearningtrialarea2_overlap/PredictedMasks'],
    [path/'TinyTestareaGAR/BingImages_zoom_19',   path/'TinyTestareaGAR/PredictedMasks_zoom_19'], 
    # EITHER!!
    #[path/'TinyTestareaGAR/BingImagesOffset_zoom_19',   path/'TinyTestareaGAR/PredictedMasksOffset_zoom_19'],
    [path/'SrokTest/BingImages_zoom_19',   path/'SrokTest/PredictedMasks_zoom_19'],
    [path/'Srok2Test/BingImages_zoom_19',   path/'Srok2Test/PredictedMasks_zoom_19'],
    #[path/'Feb10Test/BingImages_zoom_19',   path/'Feb10tTest/PredictedMasks_zoom_19'],

    # same for training data:
    [path/'TrainingData/Reservoirs_Sep10_zoom_19', path/'TrainingData/PredictedMasks_Sep10_zoom_19'],
    [path/'TrainingData/Reservoirs_Sep26_zoom_19', path/'TrainingData/PredictedMasks_Sep26_zoom_19'],
    [path/'TrainingData/Reservoirs_Dec05_zoom_19', path/'TrainingData/PredictedMasks_Dec05_zoom_19'],
    [path/'TrainingData/Reservoirs_Feb02_zoom_19', path/'TrainingData/PredictedMasks_Feb02_zoom_19'],
    [path/'TrainingData/Reservoirs_Feb10_zoom_19', path/'TrainingData/PredictedMasks_Feb10_zoom_19'],
    ### pseudo-labelled training data:
    #[path/'TrainingData/Reservoirs_TestArea3-edited_zoom_19', path/'TrainingData/PredictedMasks_TestArea3-edited_zoom_19'],
    #[path/'TrainingData/Reservoirs_4000_zoom_19', path/'TrainingData/PredictedMasks_4000_zoom_19'],
    #[path/'TrainingData/Reservoirs_Klassen_ML_April20_zoom_19', path/'TrainingData/PredictedMasks_Klassen_ML_April20_zoom_19'],
    ]

In [7]:
# path to images, i.e. which ones should be predicted?
image_path_list = [  
    ##[path/'TinyTestareaGAR/BingImagesOffset_zoom_19',   path/'TinyTestareaGAR/PredictedMasksOffset_zoom_19'],
    #[path/'SrokTest/BingImages_zoom_19',   path/'SrokTest/PredictedMasks_zoom_19'],
    [path/'GAR_large/BingImages_zoom_19',   path/'GAR_large/PredictedMasks_zoom_19'],
    ]

In [8]:
# zoom level
zoom = 19

In [9]:
# dummy functions, just needs to exist, never invoked.
def segmentron_splitter(model): return False
### mask function dummy, just needs to exist, never invoked
def get_msk(fn):  return False

In [10]:
# here we collect all model *.pkl files of our class for ensembling their predictions
import glob
ensemble_list = sorted(glob.glob(learner_path+nnet+myclass+"*"))

# intermediate models are labelled "stage" and must be excluded:
#ensemble_list = [m for m in ensemble_list if "stage" not in m ]

ensemble_list

['./models/m_deeplabv3+_C108_reservoir0.pkl']

In [11]:
# DEBUG
#ensemble_list = [ensemble_list[0]]
ensemble_list

['./models/m_deeplabv3+_C108_reservoir0.pkl']

In [12]:
import gc

# create a testset and predict 
def predict(imagelist, myclass, ensemble_list, use_TTA=False):
    no_fold = len(ensemble_list) # number of available models to iterate over
    
    firstTime = True
    # we iterate over the ensembled models:
    for fold in ensemble_list:
        #fold ="m_deeplabv3+_D02_reservoir_9"
        print ("---", fold)
        learn=load_learner(fold)
        learn.loss_func=FocalLossFlat(axis=1) ### ? ?? Needed
    
        if use_CUDA:# put it all to GPU
            dl_test = learn.dls.test_dl(imagelist).to('cuda')
            learn.model = learn.model.cuda()
            learn.dls.to('cuda')
        else:
            dl_test = learn.dls.test_dl(imagelist)

        #assert(0==1)

        if not use_TTA:
            predictions = learn.get_preds(dl=dl_test)  
        else: # with test time augmentation TTA. Better results but takes much longer
            predictions = learn.tta(dl=dl_test)
        
        # free some memory.
        del learn; gc.collect(); torch.cuda.empty_cache()
    
        # this is the ensembling, we compute the mean iteratively
        if firstTime:
            firstTime=False
            # initialization:
            preds = predictions[0]/no_fold
        else:
            # subsequently we add the other predictions and divide by the number of models
            # this results in computing the mean
            preds += predictions[0]/no_fold
       
        del predictions # free some memory.

    return preds

In [13]:
import albumentations as A
class SegmentationAlbumentationsTransform(ItemTransform):
    split_idx = 0
    def __init__(self, aug): self.aug = aug
    def encodes(self, x):
        img,mask = x
        aug = self.aug(image=np.array(img), mask=np.array(mask))
        return PILImage.create(aug["image"]), PILMask.create(aug["mask"])

In [14]:
class CombinedLoss:
    "Dice and Focal combined"
    def __init__(self, axis=1, smooth=1., alpha=1.):
        store_attr()
        self.focal_loss = FocalLossFlat(axis=axis)
        self.dice_loss =  DiceLoss(axis, smooth)
        
    def __call__(self, pred, targ):
        return self.focal_loss(pred, targ) + self.alpha * self.dice_loss(pred, targ)
    
    def decodes(self, x):    return x.argmax(dim=self.axis)
    def activation(self, x): return F.softmax(x, dim=self.axis)


In [15]:
codes = np.array(['background', myclass])
codes

array(['background', 'reservoir'], dtype='<U10')

In [16]:
%%time
if not ensemble_list:
    print("!!! NO AI MODELS !!! STOPPING")
else:
    # here comes the prediction loop:
    for image_path, mask_path in image_path_list:
        imagelist = get_image_files(image_path, recurse=False)
        #imagelist = imagelist[1499:]
        print (">> ", image_path, len(imagelist), mask_path)
        if not os.path.exists(mask_path): os.makedirs(mask_path)
            
        # here we collect the predictions from the ensemble
        preds = predict(imagelist, myclass, ensemble_list, use_TTA)
    
        # save predictions as GeoTiff images
        for i, pred in enumerate(preds):
            filename = imagelist[i].stem.replace('lidar', 'mask_') + '.tif'
            pred_scale = np.uint8((pred[1]*255).numpy())
            if CONVERT_MASK: pred_scale[pred_scale<128] = 0

            out_meta = rio.open(imagelist[i]).meta.copy()
            out_meta.update({"count": 1, # singleband greyscale
                             "compress": "LZW"}) 

            with rio.open(mask_path/filename, "w", **out_meta) as dest:
                dest.write(pred_scale, indexes=1)

>>  ../Data/GAR_large/BingImages_zoom_19 2107 ../Data/GAR_large/PredictedMasks_zoom_19
--- ./models/m_deeplabv3+_C108_reservoir0.pkl


/opt/conda/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


CPU times: user 56.9 s, sys: 9.97 s, total: 1min 6s
Wall time: 57.2 s


In [17]:
#%debug

# END

In [18]:
if not MERGE_AREAS: assert(0==1) # brute force halt:-)

### Merge reservoirs in training areas

In [19]:
from rasterio.merge import merge

In [20]:
def custom_merge(old_data, new_data, old_nodata, new_nodata, index=None, roff=None, coff=None):
    old_data[:] = np.mean([old_data, new_data], axis=0)  # <== NOTE old_data[:] updates the old data array *in place*

In [21]:
def mergeDir2mosaic(downloadDir, mergeDir):
    file_pattern =  "/r_*_*_*_" + str(zoom)+".tif"
    file_list = glob.glob(downloadDir+file_pattern)

    df = pd.DataFrame(file_list, columns=["fpath"])
    df['ID']= df['fpath'].str.split("_").str[-5]

    #print (df.head(10))
    print ('reservoirs:', df['ID'].nunique(), "snippets:", len(df))
    
    # now merge
    for i in df['ID'].unique(): 
        file_list = df[df['ID'] == i]['fpath']
        print (i, len(file_list))
        
        src_files_to_mosaic = []
        for fp in file_list:
            src = rio.open(fp)
            src_files_to_mosaic.append(src) 

        mosaic, out_trans = merge(src_files_to_mosaic,method='max')
        #mosaic, out_trans = merge(src_files_to_mosaic,method=custom_merge)
    
        out_meta = src.meta.copy() # get meta from any src
        out_meta.update({"driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_trans })
        with rio.open(mergeDir+"/"+str(i)+".tif", "w", **out_meta) as dest:
            dest.write(mosaic)

In [22]:
%%time 
for _, m in image_path_list:
    maskDir = str(m) # remove Path
    if "TrainingData" in maskDir: # this is training
        mergeDir = maskDir.replace("PredictedMasks", "PredictedReservoirs")
        if not os.path.exists(mergeDir): os.makedirs(mergeDir)
        for f in glob.glob(mergeDir+"/*"): os.remove(f)
        print (mergeDir)
        mergeDir2mosaic(maskDir, mergeDir)
        print ("---")

CPU times: user 7 µs, sys: 1e+03 ns, total: 8 µs
Wall time: 10 µs


### Merge testareas

In [23]:
#%%time 
from shutil import copyfile

for _, m in image_path_list:
    #maskDir = str(m) # remove Path
    if "Test" in str(m) or True: # this is a Testarea
        mergeFile = m.parent/'allofit_predicted.tif'
        mergeFile2 = m.parent/('allofit_predicted_'+VERSION+'.tif')

        file_list = glob.glob(str(m)+'/*.tif')
        print (m, len(file_list))
    
        # now merge
        src_files_to_mosaic = []
        for fp in file_list:
            src = rio.open(fp)
            src_files_to_mosaic.append(src) 
       
        mosaic, out_trans = merge(src_files_to_mosaic, method='max') #super easy!!!
        #mosaic, out_trans = merge(src_files_to_mosaic, method='first') #super easy!!!
        #mosaic, out_trans = merge(src_files_to_mosaic,method=custom_merge)

        out_meta = src.meta.copy()
        out_meta.update({"driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_trans,
            "compress": "lzw",})
        with rio.open(mergeFile, "w", **out_meta) as dest:
            dest.write(mosaic)
        
        copyfile(mergeFile, mergeFile2)
        print (mergeFile2)
        #break

../Data/GAR_large/PredictedMasks_zoom_19 2107
../Data/GAR_large/allofit_predicted_C108.tif
